In [299]:
from typing import List, Dict
from hazm import Lemmatizer, Normalizer, word_tokenize, stopwords_list
import pandas as pd
import hazm as hz
import pickle
import nltk

In [300]:
dataset = pd.read_csv('./datasets/Snappfood - Sentiment Analysis.csv', delimiter="\t")
dataset

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0
...,...,...,...
69995,سلام من به فاکتور غذاهایی که سفارش میدم احتیاج...,SAD,1
69996,سایز پیتزا نسبت به سفارشاتی که قبلا گذشتم کم ش...,SAD,1
69997,من قارچ اضافه رو اضافه کرده بودم بودم اما اگر ...,HAPPY,0
69998,همرو بعد ۲ساعت تاخیر اشتباه آوردن پولشم رفت رو...,SAD,1


In [301]:
dataset["label"].value_counts()

SAD      35000
HAPPY    35000
Name: label, dtype: int64

In [302]:
# there is two class of SAD and HAPPY in dataset, take 20% of data with the same class ditribution
dataset = pd.concat([dataset[dataset['label'] == 'HAPPY'].sample(frac=0.2, random_state=97), dataset[dataset['label'] == 'SAD'].sample(frac=0.2, random_state=97)]).reset_index()
dataset["label"].value_counts()

HAPPY    7000
SAD      7000
Name: label, dtype: int64

In [303]:
len(dataset.iloc[-1]["comment"])

73

In [304]:
def preprocessing_pipieline(sentence: List[str] | str ,stopwords: List[str] ,lemma: Lemmatizer,normalizer:Normalizer,index: int, dataset_len: int) -> List[str]:
    sentence = normalizer.normalize(sentence)
    sentence = hz.word_tokenize(sentence)
    acc = 0
    
    for word in sentence:
        if word in stopwords:   
            sentence.remove(word)
        word = lemma.lemmatize(word)
    percentage = 100*(index+1)/dataset_len
    if percentage % 5 == 0:
        print(f"Processing iteration {index+1}/{dataset_len} ({percentage:.0f}%)")

    return sentence

In [305]:
stopwords = hz.stopwords_list()
lemmatizer = hz.Lemmatizer()
normalizer = hz.Normalizer()
dataset["comment"] = dataset.apply(lambda row: preprocessing_pipieline(row["comment"],stopwords,lemmatizer,normalizer,row.name,len(dataset)),axis=1)

Processing iteration 700/14000 (5%)
Processing iteration 1400/14000 (10%)
Processing iteration 2100/14000 (15%)
Processing iteration 2800/14000 (20%)
Processing iteration 3500/14000 (25%)
Processing iteration 4200/14000 (30%)
Processing iteration 4900/14000 (35%)
Processing iteration 5600/14000 (40%)
Processing iteration 6300/14000 (45%)
Processing iteration 7000/14000 (50%)
Processing iteration 7700/14000 (55%)
Processing iteration 8400/14000 (60%)
Processing iteration 9100/14000 (65%)
Processing iteration 9800/14000 (70%)
Processing iteration 10500/14000 (75%)
Processing iteration 11200/14000 (80%)
Processing iteration 11900/14000 (85%)
Processing iteration 12600/14000 (90%)
Processing iteration 13300/14000 (95%)
Processing iteration 14000/14000 (100%)


In [306]:
dataset

,index,comment,label,label_id
0,45776,"[همبرگر, خوشمزه, و, پیتزا, کیفیتش, بود]",HAPPY,0
1,8003,"[خوب, فقط, سس, فراموش]",HAPPY,0
2,3124,"[عالی, واقعا, ممنونم, امیدوارم, همینطوری, بمونه]",HAPPY,0
3,27801,"[همیشه, ., …فقط, سیب, زمینی‌های, اتون, معمولی,...",HAPPY,0
4,26997,"[دقیقاااااا, سر, موقع, ۳۰, ثانیه, زودتر, شیرین...",HAPPY,0
...,...,...,...,...
13995,35802,"[رول, پای, سیب, بد, ., میکادو, کلا, خورد, تو, ...",SAD,1
13996,15602,"[مغز, کنافه‌ها, نشاسته, خمیر, نه, سرشیر, ،, شی...",SAD,1
13997,55717,"[سلام, سفارشم, ۱۲, عدد, سنگگ, معمولی, هزیته, پ...",SAD,1
13998,16481,"[افتضاح‌ترین, سوپ, قارچ, افتضاح]",SAD,1


In [307]:
file = open('./datasets/preprocessed.pkl', 'wb')
with open('./datasets/preprocessed.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [308]:
for word in hz.stopwords_list():
    if word == "تنها":
        print("yes")

yes


In [309]:
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('persian'))

[nltk_data] Downloading package stopwords to /home/vavre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


OSError: No such file or directory: '/home/vavre/nltk_data/corpora/stopwords/persian'

In [ ]:
import numpy as np
import time

random_list_of_numbers = np.random.randint(0,100,5000)

for index, number in enumerate(random_list_of_numbers):
    # pause thread for 0.01 seconds
    time.sleep(0.05)


Processing iteration 250/5000 (5%)
Processing iteration 500/5000 (10%)


KeyboardInterrupt: 